## Uniform Crime Reporting Program - FBI

Scrape data from https://ucr.fbi.gov/ucr-publications corresponding to the number of murders in certain metropolitan regions from 2006 to 2016. For each year, you will programmatically navigate to violent crimes, and then to murders, and scrape the table corresponding to Metropolitan Statistical Areas (MSAs). Along with the numerical data, you will also want to scrape and save the text descriptions on violent crimes and murders.

In [3]:
import pandas as pd
import urllib
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
"""
function
-----------
fbi_url_generator

This function pulls violent crime spreadsheets from FBI UCR website
for a given year

It takes in the year of interest and outputs a url string
"""
def fbi_url_generator(year):
    if 2006 <= year <= 2009:
        return('https://www2.fbi.gov/ucr/cius%i/data/documents/'%year +str(year)[2:]+'tbl06.xls')
    else:
        if 2010 <= year <= 2011:
            end = '/tables/table-6/output.xls'
        elif 2012 <= year <= 2013:
            end = '/tables/6tabledatadecpdf/table-6/output.xls'
        elif 2014 <= year <= 2015:
            if year == 2014:
                mid = 'Table_6_Crime_in_the_United_States_by_Metropolitan_Statistical_Area_2014/output.xls'
            else:
                mid = 'table_6_crime_in_the_united_states_by_metropolitan_statistical_area_%i.xls/output.xls' %year
            end = '/tables/table-6/%s' %mid
        elif year == 2016:
            end ='/tables/table-4/table-4/output.xls' 
        hostname = 'https://ucr.fbi.gov/crime-in-the-u.s/%i/crime-in-the-u.s.-%i' %(year, year)
        return(hostname + end)
    

In [5]:
"""
Function
--------
append_df

This function appends two dataframes

Parameters:
    input - dataframe to be appended
    output - dataframe to be appended onto
    
Returns a single dataframe 
"""
def append_df(input,output):
    if output.empty:
        output=input.copy()
    else:
        output=pd.concat([output,input])
        output.reset_index(drop='Index',inplace=True)
    return(output)

In [6]:
''' This code only needs to be run once. It pulls in all of the
    Excel Files into the Data Folder
# THIS CODE ONLY NEEDS TO BE RUN ONCE TO BRING IN ALL OF THE EXCEL FILES
version='Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36'
test=urllib.request.URLopener()
test.addheader('User-Agent',version)
for year in range(2006, 2017):
    print("Pulling: %i" %year)
    test.retrieve(url=fbi_url_generator(year),filename='data/crime_%i.xls' %year)
'''

' This code only needs to be run once. It pulls in all of the\n    Excel Files into the Data Folder\n# THIS CODE ONLY NEEDS TO BE RUN ONCE TO BRING IN ALL OF THE EXCEL FILES\nversion=\'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36\'\ntest=urllib.request.URLopener()\ntest.addheader(\'User-Agent\',version)\nfor year in range(2006, 2017):\n    print("Pulling: %i" %year)\n    test.retrieve(url=fbi_url_generator(year),filename=\'data/crime_%i.xls\' %year)\n'

In [60]:
#Define empty dataframes
population=pd.DataFrame()
msa_description=pd.DataFrame()
total_crime_by_MSA=pd.DataFrame()
crime_rate_by_MSA=pd.DataFrame()
crime_by_city_in_MSA=pd.DataFrame()

In [19]:
df_allyears = pd.DataFrame()
for year in range(2006, 2017):
    df = pd.read_excel("data/crime_%i.xls" %year,skiprows=[0,1],header=1)

    #######
    # NOTE - misc column has msa population, city population and estimate percentage
    ######
    df=df.iloc[:,0:12] 
    df.columns=['MSA', 'counties','misc', 'violent_crime','mur_mans', 'rape', 'robbery',
                'assault', 'property', 'burglary', 'larceny','mv_theft']

    # Drop footnotes
    footnotes = df['MSA'].str[0].str.isdigit().fillna(False)
    df = df.loc[~footnotes, :]

    # Get rid of numbers in MSA
    df['MSA'] = df['MSA'].str.replace('\d+', '')
    # Set empty columns to NaN for MSA
    df["MSA"] = df["MSA"].replace(' ', np.nan, regex=False)
    
    # Sometimes city  names get put in MSA column
    # Messes up carry forward
    df.loc[df['MSA'].str.contains("City of").fillna(False), "MSA"] = np.nan

    # Carry MSA name forward to fill in for all cells
    df.loc[:,'MSA'] = df.loc[:, 'MSA'].fillna(method='ffill')

    ##############
    # POPULATION - grab population and fill in for all MSA
    ##############
    pop_row = df.counties.isnull()
    pop = df.loc[pop_row, ["MSA", 'misc']]
    pop = pop.rename(index=str, columns={'misc': 'msa_pop'})
    
    # Merge population back in
    df = df.loc[~pop_row, :]
    df = df.merge(pop, how='outer', on='MSA')

    ################
    # Descriptions - don't need county descriptions 
    ################
    df = df.loc[df.counties.str.contains("Includes") == False, :]


    ###########################################
    # GOING LONG TO WIDE FOR CRIME VARIABLES
    ###########################################
    crime_vars = ['violent_crime','mur_mans', 'rape', 'robbery',
                  'assault', 'property', 'burglary', 'larceny','mv_theft']

    #########
    # CITIES
    #########
    city_vars = ['MSA', 'counties', 'misc'] + crime_vars
    # Split data Frame
    cities = df.counties.str.contains("City")
    city_df = df.loc[cities, city_vars]
    city_df = city_df.rename(index=str, columns={'misc': 'city_pop'})
    
    # Grab largest city for each MSA and merge back on
    city_df = city_df.sort_values(['MSA','city_pop'], ascending=False)
    large_city = city_df.groupby('MSA').first().reset_index()

    # Rename crime variables to denote city only crime 
    large_city.columns = ['MSA', 'counties', 'city_pop'] + ['city_' + i for i in crime_vars]
    large_city = large_city.rename(index=str, columns={'counties':'largest_city'})
    # Get rid of "City of"
    large_city.loc[:,'largest_city'] = large_city.loc[:, 'largest_city'].str.replace('City of','')
    
    # Merge back to main dataframe
    df = df.loc[~cities, ]
    df = df.merge(large_city, how='outer', on='MSA')

    ###############
    # CRIME RATE
    ###############
    rates = df.counties.str.contains("Rate per")
    rate_vars = ['MSA'] + crime_vars
    rates_df = df.loc[rates, rate_vars]
    rates_df.columns = ['MSA'] + ['rate_' + i for i in crime_vars]

    df = df.loc[~rates, :]
    df = df.merge(rates_df, how='outer', on='MSA')

    ########################
    # MSA-WIDE CRIME STATS
    ########################

    # If the entire MSA reported then there is just one row of numbers
    # If the entire MSA did not report, then there are two rows
            # first row is areas that reported
            # second report is an estimated total
    # We are going to grab the estimates total so our data
    # reflects all areas for all MSA

    # Create Flag for those that do not have complete coverage
    # and are thus estimates
    mins = df.groupby('MSA').misc.min().reset_index()
    mins.columns = ['MSA', 'min_coverage']
    df = df.merge(mins, how='outer', on='MSA')
    df['estimate'] = 0
    df.loc[df.min_coverage < 1, 'estimate'] = 1
    del df['min_coverage']

    # Now only keeping rows with coverage = 1
    # will either be all area or the estimate for all area
    df = df.loc[df.misc == 1, :]

    # Now no longer need coverage or whether its estimate or not
    del df['misc']
    del df['counties']
    
    df['year'] = year
    
    # Append to existing Frame
    df_allyears = append_df(df, df_allyears)

In [20]:
df_allyears = df_allyears.sort_values(["MSA", 'year'])
print(df_allyears.columns)
df_allyears.head()

Index(['MSA', 'violent_crime', 'mur_mans', 'rape', 'robbery', 'assault',
       'property', 'burglary', 'larceny', 'mv_theft', 'msa_pop',
       'largest_city', 'city_pop', 'city_violent_crime', 'city_mur_mans',
       'city_rape', 'city_robbery', 'city_assault', 'city_property',
       'city_burglary', 'city_larceny', 'city_mv_theft', 'rate_violent_crime',
       'rate_mur_mans', 'rate_rape', 'rate_robbery', 'rate_assault',
       'rate_property', 'rate_burglary', 'rate_larceny', 'rate_mv_theft',
       'estimate', 'year'],
      dtype='object')


,MSA,violent_crime,mur_mans,rape,robbery,assault,property,burglary,larceny,mv_theft,...,rate_mur_mans,rate_rape,rate_robbery,rate_assault,rate_property,rate_burglary,rate_larceny,rate_mv_theft,estimate,year
0,"Abilene, TX M.S.A.",638,6,75,109,448,5741,1531,3852,358,...,3.7,46.1,67,275.2,3526.9,940.6,2366.4,219.9,0,2006
357,"Abilene, TX M.S.A.",748,10,103,173,462,5657,1606,3708,343,...,6.3,65.3,109.7,292.9,3586.9,1018.3,2351.1,217.5,0,2007
721,"Abilene, TX M.S.A.",716,7,95,188,426,5124,1417,3443,264,...,4.4,59.7,118,267.5,3217.4,889.8,2161.9,165.8,0,2008
1081,"Abilene, TX M.S.A.",741,7,134,146,454,5607,1488,3816,303,...,4.4,83.9,91.5,284.4,3512.5,932.1,2390.5,189.8,0,2009
1459,"Abilene, TX M.S.A.",675,5,78,116,476,5772,1610,3925,237,...,3.1,48.9,72.7,298.3,3617.3,1009,2459.8,148.5,0,2010


In [4]:


#Define empty dataframes
population=pd.DataFrame()
msa_description=pd.DataFrame()
total_crime_by_MSA=pd.DataFrame()
crime_rate_by_MSA=pd.DataFrame()
crime_by_city_in_MSA=pd.DataFrame()

for year in range(2006,2017):
    #Fetch the xls file from the web server
    test.retrieve(url=fbi_url_generator(year),filename='output.xls')

    #Read xls file into a dataframe. Exclude the first two rows since they are part of the title
    input_df=pd.read_excel("output.xls",skiprows=[0,1],header=1)
    #Drop unwanted columns and rename columns
    input_df=input_df.iloc[:,0:12] 
    input_df.columns=['Metropolitan Statistical Area', 'Counties/principal cities','Population', 'Violent crime','Murder and nonnegligent manslaughter', 'Rape1', 'Robbery','Aggravated assault', 'Property crime', 'Burglary', 'Larceny-theft','Motor vehicle theft']
    #Add year
    input_df['year']=year
    
    #POPULATION EXTRACTION
    population_stg=input_df[input_df['Metropolitan Statistical Area'].notnull()].loc[:,['Metropolitan Statistical Area','Population','year']]
    population_stg.dropna(how='any',inplace=True)         #Remove any rows without a population
    population_stg.reset_index(drop='Index',inplace=True) #Reset index
    #Merge Results
    population=df_merger(population_stg,population) 
    #Fill all cells of MSA
    input_df['Metropolitan Statistical Area']=input_df['Metropolitan Statistical Area'].fillna(method='ffill')
    #Drop population lines
    input_df.drop(input_df[(input_df['Counties/principal cities'].isnull()) & (input_df['Population'].notnull())].index,inplace=True)
    
    #MSA DESCRIPTION EXTRACTION
    msa_description_stg=input_df[input_df['Counties/principal cities'].str.startswith('Includes', na=False)].loc[:,['Metropolitan Statistical Area','Counties/principal cities']]
    msa_description_stg.reset_index(drop='Index',inplace=True)
    #Merge Results
    msa_description=df_merger(msa_description_stg,msa_description)
    #Drop Description rows
    input_df.drop(input_df[input_df['Counties/principal cities'].str.startswith('Includes', na=False)].index,inplace=True)
    
    #REMOVE FOOTNOTES
    input_df.drop(input_df[input_df['Metropolitan Statistical Area'].str[0].str.isdigit()].index,inplace=True)
    
    #TOTAL CRIME BY MSA EXTRACTION
    total_crime_by_MSA_stg=input_df[input_df['Population']==1]
    total_crime_by_MSA_stg['Total vs Estimated']=total_crime_by_MSA_stg['Counties/principal cities'].str[0]
    total_crime_by_MSA_stg.drop(['Population','Counties/principal cities'], axis=1,inplace=True)
    total_crime_by_MSA_stg.reset_index(drop='Index',inplace=True)
    total_crime_by_MSA=df_merger(total_crime_by_MSA_stg,total_crime_by_MSA)
    input_df.drop(input_df[input_df['Counties/principal cities'].str.startswith(('Total','Estimated'), na=False)].index,inplace=True)

    #CRIME RATE BY MSA EXTRACTION    
    crime_rate_by_MSA_stg=input_df[input_df['Counties/principal cities'].str.startswith(('Rate'), na=False)]
    crime_rate_by_MSA_stg.drop(['Population','Counties/principal cities'], axis=1,inplace=True)
    crime_rate_by_MSA_stg.reset_index(drop='Index',inplace=True)
    crime_rate_by_MSA=df_merger(crime_rate_by_MSA_stg,crime_rate_by_MSA)
    input_df.drop(input_df[input_df['Counties/principal cities'].str.startswith(('Rate'), na=False)].index,inplace=True)
    
    #CRIME BY CITIES WITHIN MSA EXTRACTION 
    crime_by_city_in_MSA=df_merger(input_df,crime_by_city_in_MSA)

In [5]:
population.head(4)

,Metropolitan Statistical Area,Population,year
0,"Abilene, TX M.S.A.1",162776,2006
1,"Albany, GA M.S.A.",168071,2006
2,"Albany-Schenectady-Troy, NY M.S.A.",851151,2006
3,"Albuquerque, NM M.S.A.",808790,2006


In [6]:
msa_description.drop_duplicates(inplace=True)
msa_description.sort_values('Metropolitan Statistical Area').head(4)

,Metropolitan Statistical Area,Counties/principal cities
355,"Abilene, TX M.S.A.","Includes Callahan, Jones, and Taylor Counties"
0,"Abilene, TX M.S.A.1","Includes Callahan,1 Jones, and Taylor Counties"
2580,"Aguadilla-Isabela, Puerto Rico M.S.A.","Includes Aguada, Aguadilla, Anasco, Isabela, L..."
2963,"Aguadilla-Isabela, Puerto Rico M.S.A.","Includes Aguada, Aguadilla, Anasco, Isabela, L..."


In [7]:
total_crime_by_MSA.head(4)

,Metropolitan Statistical Area,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,year,Total vs Estimated
0,"Abilene, TX M.S.A.1",638,6,75,109,448,5741,1531,3852,358,2006,T
1,"Albany, GA M.S.A.",676,11,37,265,363,7171,2162,4494,515,2006,E
2,"Albany-Schenectady-Troy, NY M.S.A.",3165,18,207,964,1976,23349,4849,17314,1186,2006,T
3,"Albuquerque, NM M.S.A.",6291,72,420,1410,4389,39023,8972,23336,6715,2006,E


In [8]:
crime_rate_by_MSA.head(4)

,Metropolitan Statistical Area,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,year
0,"Abilene, TX M.S.A.1",391.9,3.7,46.1,67,275.2,3526.9,940.6,2366.4,219.9,2006
1,"Albany, GA M.S.A.",402.2,6.5,22,157.7,216,4266.6,1286.4,2673.9,306.4,2006
2,"Albany-Schenectady-Troy, NY M.S.A.",371.8,2.1,24.3,113.3,232.2,2743.2,569.7,2034.2,139.3,2006
3,"Albuquerque, NM M.S.A.",777.8,8.9,51.9,174.3,542.7,4824.9,1109.3,2885.3,830.3,2006


In [9]:
crime_by_city_in_MSA.head(4)

,Metropolitan Statistical Area,Counties/principal cities,Population,Violent crime,Murder and nonnegligent manslaughter,Rape1,Robbery,Aggravated assault,Property crime,Burglary,Larceny-theft,Motor vehicle theft,year
0,"Abilene, TX M.S.A.1",City of Abilene,118009,554,5,67,107,375,5045,1282,3460,303,2006
1,"Albany, GA M.S.A.",City of Albany,77815,553,8,31,243,271,5279,1645,3235,399,2006
2,"Albany-Schenectady-Troy, NY M.S.A.",City of Albany,93773,1217,5,50,388,774,4820,1058,3521,241,2006
3,"Albany-Schenectady-Troy, NY M.S.A.",City of Schenectady,61444,712,6,52,309,345,3449,1119,1994,336,2006


### Next Steps:

1. Cleanse the MSA Description table
2. Derive State and MSA Names
3. Nan Imputation
